In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.executing_eagerly()

In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)
## Imports libs

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [ ]:
path1 = '/home/maxwellsam/Multimodal-COVID19-Images/generatedImages/COVID19/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/Multimodal-COVID19-Images/generatedImages/NonCOVID19/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

In [ ]:
path3 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_COVID/'
prefix3 = 'ct_covid'
path4 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_NonCOVID/'
prefix4 = 'ct_noncovid'

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covnid-19 negative

In [ ]:
ct_covid_features_df_shap =  processImages(path3,1)#1--> covid-19 positive
ct_noncovid_features_df_shap =  processImages(path4,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
cvd_imgs_shap = [ct_noncovid_features_df_shap, ct_covid_features_df_shap]
cvd_imgs_dataset_shap = pd.concat(cvd_imgs_shap)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap = cvd_imgs_dataset_shap.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap)

#### Prepare negative covid images for machine learning ready

In [ ]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_negative_only = input_data_x_
output_label_y_negative_only = output_label_y_
print('Input_x Data Shape: \n{0}'.format(input_data_x_negative_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_negative_only.shape))

#### Prepare positive covid images for machine learning ready

In [ ]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_positive_only = input_data_x__
output_label_y_positive_only = output_label_y__
print('Input_x Data Shape: \n{0}'.format(input_data_x_positive_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_positive_only.shape))

#### Prepare both positive and negetive covid images for machine learning ready

In [ ]:
input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))

In [ ]:
cvd_imgs_dataset

#### Prepare both positive and negetive covid shap images for machine learning ready

In [ ]:
input_data_x_shap = cvd_imgs_dataset_shap.iloc[:,:-1].to_numpy().reshape((3718,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap['class_label'])
cvd_imgs_dataset_shap
cvd_imgs_dataset_shap = pd.get_dummies(cvd_imgs_dataset_shap, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_shap = np.array(cvd_imgs_dataset_shap[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x_shap.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_shap.shape))

In [ ]:
cvd_imgs_dataset_shap

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    input_data_x, output_label_y, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap, test_features_shap, train_labels_shap, test_labels_shap = train_test_split(
    input_data_x_shap, output_label_y_shap, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_negative_only, test_features_negative_only, train_labels_negative_only, test_labels_negative_only = train_test_split(
    input_data_x_negative_only, output_label_y_negative_only, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_positive_only, test_features_positive_only, train_labels_positive_only, test_labels__positive_only = train_test_split(
    input_data_x_positive_only, output_label_y_positive_only, test_size=.20, random_state=42)

In [ ]:
train_features_shap.shape

In [ ]:
train_labels

In [ ]:
train_features.shape

In [ ]:
image_size = 200
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Normalization(),
        tf.keras.layers.Resizing(image_size, image_size),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(factor=0.02),
        tf.keras.layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(train_features)

### Building the multimodal

In [ ]:
def CNN_Mod(inputShape, num_hl, filters_, kernel_size_, stride_poolSize, hl_conv_activation, hl_list, dropout_val, ol_activation):
    '''
    CNN_Mod(model, num_hl, hl_list, hl_conv_activation, ol_activation, dropout_val, inputShape, filters_, kernel_size_, stride_poolSize):
        model            = CNN Model
        num_hl           = Number of hidden layers
        hl_list          = List of hidden layer units
        hl_conv_activation = Activation function for convolutional layers
        ol_activation    = Activation function for the output layer
        dropout_val      = List of dropout values for hidden layers
        inputShape       = Shape of the input data (e.g., (300, 300, 1))
        filters_         = Number of filters for Conv layers
        kernel_size_     = Size of the kernel (height, width)
        stride_poolSize  = Stride and pool size (same value for both)
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    
    inputs = tf.keras.layers.Input(shape=inputShape)

    # Augment data (assuming data_augmentation is predefined)
    augmented = data_augmentation(inputs)
    
    conv_layer_1 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(augmented)
    conv_layer_1_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)

    # Convolutional Layer 2 with Max-Pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters=filters_, kernel_size=kernel_size_, 
                                          strides=stride_poolSize, activation=hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPooling2D(pool_size=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_2_max_pool)

    # Flattening the output of the last convolutional block
    flatten_layer = tf.keras.layers.Flatten()(x2)

    # Adding Dense and Dropout Layers
    dense_layers = flatten_layer
    for i in range(num_hl):
        dense_layers = tf.keras.layers.Dense(units=hl_list[i], activation=hl_conv_activation)(dense_layers)
        dense_layers = tf.keras.layers.Dropout(dropout_val[i])(dense_layers)

    # Create the CNN model
    model = tf.keras.Model(inputs=inputs, outputs=dense_layers)
    return model

# Define the multimodal model
def Multimodal_CNN(inputShape1, inputShape2, num_hl, shared_label_units, 
                   filters_, kernel_size_, stride_poolSize, hl_conv_activation, hl_list, dropout_val, ol_activation):
    # Create two branches
    branch1 = CNN_Mod(inputShape1, num_hl, filters_, kernel_size_, stride_poolSize, hl_conv_activation, hl_list, dropout_val, ol_activation)
    branch2 = CNN_Mod(inputShape2, num_hl, filters_, kernel_size_, stride_poolSize, hl_conv_activation, hl_list, dropout_val, ol_activation)

    # Combine outputs from both branches
    combined = tf.keras.layers.Concatenate()([branch1.output, branch2.output])

    # Add shared dense layers after combination
    x = tf.keras.layers.Dense(units=shared_label_units, activation=hl_conv_activation)(combined)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    # Output layer with shared label
    final_output = tf.keras.layers.Dense(units=2, activation=ol_activation)(x)

    # Define the multimodal model
    multimodal_model = tf.keras.Model(inputs=[branch1.input, branch2.input], outputs=final_output)
    return multimodal_model

# Compile the model
def compile_multimodal(model, learning_rate, loss_, metrics_):
    '''
    def compile_model(model, loss_, learningRate, metrics_):
    model: the model to compile
    loss_: the loss function
    learningRate: learning rate
    metrics_: list of metrics to track (e.g., ['accuracy'])
    '''
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss_, metrics=metrics_)
    return model

# Train the multimodal model
def train_multimodal(model, val_split_size, batch_size_, numEpochs, patience_, monitor_, mode):
    
    history = model.fit(
        x=[train_features[:2974], train_features_shap],
        y=train_labels,
        validation_split = val_split_size,
        batch_size=batch_size_,
        epochs=numEpochs,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor=monitor_, patience=patience_, verbose=1, mode=mode)
        ]
    )
    return history

def evaluateModel_loss(history):
    print()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Training loss','Validation loss'], loc = 'upper left')
    plt.savefig("Training_validation_loss_with_i.jpg")
    plt.show()
    plt.close()
    #plt.show()
    print()
def evaluateModel_accuracy(history):
    print()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Training and Validation Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend(['Training Accuracy','Validation Accuracy'], loc = 'upper left')
    #plt.show()
    plt.savefig("Training_validation_accuracy_with_i.jpg")
    plt.show()
    plt.close()
    print()

print(CNN_Mod.__doc__)


In [ ]:
inputShape1 = (300, 300, 1) 
inputShape2 = (300, 300, 1)  
filters_ = 100
kernel_size_ = (5, 5)
stride_poolSize = 2
hl_conv_activation = 'relu'
hl_list = [288, 128, 33, 11]  
dropout_val = [0.00,0.35,0.00,0.00]
ol_activation = 'softmax'
shared_label_units = 64
learning_rate = 0.0015272304174499124
loss_ = 'categorical_crossentropy'
metrics_ = ['accuracy']
batch_size = 128
epochs = 100
val_split = 0.2
patience = 10
monitor = 'val_loss'
mode = 'min'
num_hl = 4

In [ ]:
multimodal_model = Multimodal_CNN(inputShape1, inputShape2, num_hl, shared_label_units, 
                                  filters_, kernel_size_, stride_poolSize, hl_conv_activation, 
                                  hl_list, dropout_val, ol_activation)
compiled_model = compile_multimodal(multimodal_model, learning_rate, loss_, metrics_)

history = train_multimodal(compiled_model,val_split,batch_size,epochs,patience,monitor,mode)

In [ ]:
evaluateModel_loss(history)
evaluateModel_accuracy(history)

In [ ]:
def getCorrectLabel(x):
        if x == 0:
            return 'Non-Covid'
        if x == 1:
            return 'Covid'
        
def D1_to_D2(y_pred):
    y_true = {'y_true':[i[0] for i in y_pred]}
    y_true_df = pd.DataFrame(y_true)
    y_true_df = pd.get_dummies(y_true_df, columns =['y_true'])
    output_label_y = np.array(y_true_df[y_true_df.columns])
    return output_label_y

def getModelEvaluation(model, test_features, test_features_shap, test_labels):
    inputs = [test_features, test_features_shap]
    predicted_labels = model.predict(inputs)
    df = pd.DataFrame(predicted_labels)
    df['Predicted_Labels'] = np.array(df.iloc[:,:2]).argmax(axis =1)
    df['Predicted_Labels'] = df['Predicted_Labels'].apply(getCorrectLabel)
    if (test_labels.shape[1]==2):
        df['Actual_Labels'] = test_labels[:,:2].argmax(axis = 1)
    else:
        df['Actual_Labels'] = D1_to_D2(test_labels)[:,:2].argmax(axis = 1)
    df['Actual_Labels'] = df['Actual_Labels'].apply(getCorrectLabel)
    # df['Probality'] = np.max(np.array(df.iloc[:,:7]))
    #df.drop([0,1,2,3,4,5,6,7,8,9],axis =1,inplace=True)
    #print(df.head(100))
    cm = pd.crosstab(df.Predicted_Labels, df.Actual_Labels)
    fig = plt.figure(figsize=(20,5))
    ax1 = plt.subplot(121)
    sns.heatmap(cm,annot = True,cmap='Blues')
    ax1.set_title('')
    # Saving the figure.
    plt.savefig("test_Confusion_Matrix_with_i.jpg")
    plt.show()
    plt.close()

    pred = model.evaluate([test_features, test_features_shap],test_labels)
    print("loss = " + str(pred[0]))
    print("test accuracy = " + str(pred[1]))

In [ ]:
label_predicted = multimodal_model.predict((test_features[:744], test_features_shap))

In [ ]:
n_samples = 10

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    sample_image = test_features[index].reshape(300, 300)
    plt.imshow(sample_image, cmap="binary")
    plt.title("Label:" + str(test_labels[index].argmax()))
    plt.axis("off")

plt.show()

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    #sample_image = image_predicted[index].reshape(300, 300)
    #plt.imshow(sample_image, cmap="binary")
    plt.title("Predicted:" + str(np.argmax(label_predicted[index])))
    plt.axis("off")

plt.show()

#### Test on Original dataset

In [ ]:
import seaborn as sns
getModelEvaluation(multimodal_model,test_features,test_features_shap,test_labels)#test_features_negative_onlyshap_, train_labels_shap_, test_labels_shap_